# Tests



In [1]:
import molsysmt as msm
from molsysmt import systems

In [2]:
from molsysmt.native import H5MSMFileHandler

In [4]:
molsys = msm.systems['chicken villin HP35']['traj_chicken_villin_HP35_solvated.h5msm']

In [5]:
handler = H5MSMFileHandler(molsys)

In [25]:
handler.file['topology']['atoms']['name'].shape[0]

4456

In [ ]:
pdb_file_handler.entry.crystallographic_and_coordinate_transformation.scale.u2